In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
os.environ['PYTHONHASHSEED']='0'

In [ ]:
np.random.seed(73)

In [ ]:
import random as rn
rn.seed(104)

In [ ]:
import pandas as pd 
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def data_refresh():
 train=pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
 y_train=pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
 sample=pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")
 test=pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
 y_train.drop("sig_id",axis=1,inplace=True)
 train.drop(["sig_id","cp_type"],axis=1,inplace=True)
 train["cp_dose"]=pd.get_dummies(train["cp_dose"])["D1"]
 train["cp_time"]=(train["cp_time"]-24)/72
 return train,y_train

In [ ]:


from keras.models import Sequential
from keras.layers import Dense,Dropout
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense,Dropout
import tensorflow as tf
def build_model_negative(hp):  
  model = Sequential([
    Dense(
        units=hp.Int('units_1', min_value=1024, max_value=2048, step=128),
        activation=hp.Choice('act_1',values=["relu","tanh","sigmoid"])
        ,input_dim=874,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=100)),
    Dropout(
        rate=hp.Choice('conv_2_kernel', values = [0.3,0.4,0.5,0.66,0.77])
    ),
      Dense(
        units=hp.Int('units_1', min_value=512, max_value=1024, step=128),
        activation=hp.Choice('act_1',values=["relu","tanh","sigmoid"]),
        kernel_initializer=tf.keras.initializers.GlorotUniform(seed=102)),
    Dropout(
        rate=hp.Choice('conv_2_kernel', values = [0.3,0.4,0.5,0.66,0.77])
    ),
      Dense(
        units=hp.Int('units_1', min_value=512, max_value=1024, step=16),
        activation=hp.Choice('act_1',values=["relu","tanh","sigmoid"])
      ,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=103)),
    Dropout(
        rate=hp.Choice('conv_2_kernel', values = [0.3,0.4,0.5,0.66,0.77])
    ),
      
    Dense(
        units=hp.Int('dense_1_units', min_value=256, max_value=512, step=8),
        activation=hp.Choice('act2',values=["relu","tanh","sigmoid"])
    ,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=78)),Dropout(
        rate=hp.Choice('conv_2_kernel', values = [0.3,0.4,0.5,0.66,0.77])
    ),
      Dense(
        units=hp.Int('dense_1_units', min_value=256, max_value=512, step=8),
        activation=hp.Choice('act2',values=["relu","tanh","sigmoid"])
    
      ,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=56)),
      
    Dense(206, activation='sigmoid',kernel_initializer=tf.keras.initializers.GlorotUniform(seed=89))])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='binary_crossentropy')
  
  return model






In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
uner_search=RandomSearch(build_model_negative,
                          objective='val_loss',
                          max_trials=50,project_name="kaggle")

In [ ]:
train,y_train=data_refresh()
uner_search.search(train,y_train,epochs=10,validation_split=0.18)

In [ ]:
model_0=uner_search.get_best_models(num_models=3)[0]
model_1=uner_search.get_best_models(num_models=3)[0]
model_2=uner_search.get_best_models(num_models=3)[0]
model_3=uner_search.get_best_models(num_models=3)[0]

In [ ]:
for i in uner_search.get_best_models(num_models=3)[0].layers:
    print(i.get_config())

In [ ]:
model_0.summary()

In [ ]:
train,y_train=data_refresh()
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train, y_train, test_size=0.18, random_state=85)
model_0.fit(X_train,Y_train,epochs=50,callbacks=[tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=6,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)],batch_size=512,validation_data=(X_test,Y_test))
train,y_train=data_refresh()
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train, y_train, test_size=0.18, random_state=123)
model_1.fit(X_train,Y_train,epochs=50,callbacks=[tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=6,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)],batch_size=256,validation_data=(X_test,Y_test))
train,y_train=data_refresh()
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train, y_train, test_size=0.18, random_state=23)
model_2.fit(X_train,Y_train,epochs=50,callbacks=[tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=6,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)],batch_size=256,validation_data=(X_test,Y_test))
train,y_train=data_refresh()
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train, y_train, test_size=0.18, random_state=43)
model_3.fit(X_train,Y_train,epochs=50,callbacks=[tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=6,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)],batch_size=256,validation_data=(X_test,Y_test))

In [ ]:
#batch_size=256
import matplotlib.pyplot as plt
plt.plot(model_0.history.history["val_loss"],color="green",marker="o")
plt.plot(model_1.history.history["val_loss"],color="blue",marker="o")
plt.plot(model_2.history.history["val_loss"],color="black",marker="o")
plt.plot(model_3.history.history["val_loss"],color="red",marker="o")

In [ ]:
#batch_size=512
import matplotlib.pyplot as plt
plt.plot(model_0.history.history["val_loss"],color="green",marker="o")
plt.plot(model_1.history.history["val_loss"],color="blue",marker="o")
plt.plot(model_2.history.history["val_loss"],color="black",marker="o")
plt.plot(model_3.history.history["val_loss"],color="red",marker="o")

In [ ]:
#batch_size=1024,epochs=30
plt.plot(model_0.history.history["val_loss"],color="green",marker="o")
plt.plot(model_1.history.history["val_loss"],color="blue",marker="o")
plt.plot(model_2.history.history["val_loss"],color="black",marker="o")
plt.plot(model_3.history.history["val_loss"],color="red",marker="o")

In [ ]:
#batch_size=512,epochs=50
plt.plot(model_0.history.history["val_loss"],color="green",marker="o")
plt.plot(model_1.history.history["val_loss"],color="blue",marker="o")
plt.plot(model_2.history.history["val_loss"],color="black",marker="o")
plt.plot(model_3.history.history["val_loss"],color="red",marker="o")

In [ ]:
plt.plot(model_0.history.history["loss"],color="green",marker="o")
plt.plot(model_1.history.history["loss"],color="blue",marker="o")
plt.plot(model_2.history.history["loss"],color="black",marker="o")
plt.plot(model_3.history.history["loss"],color="red",marker="o")

In [ ]:
def data_test_refresh():
    test=pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
    test.drop(["sig_id","cp_type"],axis=1,inplace=True)
    test["cp_dose"]=pd.get_dummies(test["cp_dose"])["D1"]
    test["cp_time"]=(test["cp_time"]-24)/72
    return test

In [ ]:
test=data_test_refresh()
x1=model_0.predict(test)
x2=model_1.predict(test)
x3=model_2.predict(test)
x4=model_3.predict(test)

In [ ]:
test=pd.DataFrame(x1)+pd.DataFrame(x2)+pd.DataFrame(x3)+pd.DataFrame(x4)

In [ ]:
test=test/4

In [ ]:
sample=pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")

In [ ]:
sample

In [ ]:
test=np.array(test)
test_avg=pd.DataFrame(test,columns=sample.columns[1:])
sample[sample.columns[1:]]=test_avg
sample.to_csv("submission.csv",index=False)


In [ ]:
sample